# Link prediction in machine learning open-source quasi network 

This project seeks to derive promising technologies in the GitHub machine learning ecosystem by using GNN-based link prediction techniques.

The network to be used for analysis is the topic quasi network in the GitHub open source. This is a topic quasi network derived from a heterogeneous network based on the relationship between projects in rows, topics in columns, and topic appearances as components of the network.
 
Previously, it was attempted to apply direct link prediction to a heterogeneous network of 'project-topic', but this was put off for follow-up research.

In [1]:
# Basic data analysis and data IO
import numpy as np
import pandas as pd
import pickle

# Network preprocssing 
import scipy.sparse as sp
import torch

# GNN 
from torch_geometric.data import InMemoryDataset, Data
from torch_geometric.nn import GCNConv, VGAE
import torch_geometric.transforms as T 
from torch_geometric.utils import negative_sampling

# Evaluation 
from sklearn.metrics import roc_auc_score, f1_score, precision_score, recall_score

# Clustering
from sklearn.cluster import KMeans
from collections import Counter

# Visualization
from sklearn.manifold import TSNE
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import chart_studio
from chart_studio.plotly import plot

# ETC
from tqdm import tqdm
import networkx as nx
import warnings 

warnings.filterwarnings(action='ignore')
chart_studio.tools.set_credentials_file(username='Your User Name', api_key='Your API Key')

# 1. Construct Data loader 

In [2]:
node_feature = None

In [3]:
"""
- Select feature information to be input to the GNN algorithm
- Mainly divided into two types: None and basic
     - None: contains no information, identity matrix as input, total length 39
     - basic : readme information 
"""

original = pd.read_csv('data/network/coword/coword.csv', index_col=0)
node_data = original.columns
n_topic = 6463

G_original = nx.from_pandas_adjacency(original)
print(nx.info(G_original))


def feature_selector(network, feature=None) :
    if feature == 'basic' :
        
        # call document embedding vector
        with open('data/readme/2048_readme_vector.pickle', 'rb') as f :
            doc2vec = pickle.load(f)
        repo_feature = torch.tensor(doc2vec).to(torch.float)
        topic_feature = torch.zeros((n_topic, repo_feature.size(1))).to(torch.float)
        x = torch.cat((topic_feature, repo_feature), 0)
        
        # not use in this project 
        '''
        data = pd.read_csv('data/data/filtered_data.csv')

        # call social metric 
        need_feature = ['contributors_count', 'stargazers_count', 'forks_count', 'watchers_count']
        feature_data = data[need_feature]

        x = torch.tensor(feature_data.values).to(torch.float)
        x = torch.cat((x, doc2vec), 1)
        '''
        

    # add all genereated feature vectors
    elif feature == None :
        x = torch.eye(network.shape[0]).to(torch.float)
        
    return x

Graph with 6463 nodes and 73527 edges


In [4]:
class MyData(InMemoryDataset) :
    def __init__(self, transform) :
        super(MyData, self).__init__()

        network = pd.read_csv('data/network/coword/coword.csv', index_col=0)
        self.node_name = network.columns
        network = network.values

        adj = sp.csr_matrix(network).tocoo()
        row = torch.from_numpy(adj.row).to(torch.float)
        col = torch.from_numpy(adj.col).to(torch.float)

        edge_index = torch.stack([row, col], dim=0).to(torch.long)
        x = feature_selector(network, feature=node_feature)
        edge_weight = torch.tensor(adj.data).to(torch.float)

        data = Data(x=x, edge_index=edge_index, edge_attr=edge_weight)
        if transform : 
            data = transform(data)

        self.data, self.slices = self.collate([data])

- Slices contain shape information of data, but there is no need to use it.
- All information is included in the data, so you can just use it

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


transform_train = T.Compose([
    T.NormalizeFeatures(),
    T.ToDevice(device),
    T.RandomLinkSplit(num_val=0.05, num_test=0.1, is_undirected=False, split_labels=True,
                      add_negative_train_samples=True),
])

transform_embedding = T.Compose([
    T.NormalizeFeatures(),
    T.ToDevice(device),
])

# train splited data 
data_load = MyData(transform=transform_train)
train_data, val_data, test_data = data_load[0]

# embedding full data 
full_data = MyData(transform=transform_embedding)
data = full_data.data

- As a result of loading data, 4 basic information is included
    - x : original network, sparse network type
    - edge_index : A variant of the sparse network, including only pairs of indexes in which values ​​exist
    - edge_attribute : edge weight
    - train, val, test mask: indicates the purpose of each edge <br></br>

- Train, val, test split edge index, edge weight information is also included

---

# 2. Model construction

In [ ]:
class GCNmodel(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def encode(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        return self.conv2(x, edge_index)

    def decode(self, z, edge_label_index):
        return (z[edge_label_index[0]] * z[edge_label_index[1]]).sum(dim=-1)

    def decode_all(self, z):
        prob_adj = z @ z.t()
        return prob_adj

---

# 3. GCN based link prediction

In [ ]:
device = 'cpu'
hidden_size = 128
embedding_size = 64
epochs = 200
iteration = 1000

model = GCNmodel(data_load.num_features, hidden_size, embedding_size).to(device)
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.01)
criterion = torch.nn.BCEWithLogitsLoss()

In [ ]:
def train():
    model.train()
    optimizer.zero_grad()
    z = model.encode(train_data.x, train_data.edge_index)

    # We perform a new round of negative sampling for every training epoch:
    neg_edge_index = negative_sampling(
        edge_index=train_data.edge_index, num_nodes=train_data.num_nodes,
        num_neg_samples=train_data.edge_label_index.size(1), method='sparse')

    edge_label_index = torch.cat(
        [train_data.edge_label_index, neg_edge_index],
        dim=-1,
    )
    edge_label = torch.cat([
        train_data.edge_label,
        train_data.edge_label.new_zeros(neg_edge_index.size(1))
    ], dim=0)

    out = model.decode(z, edge_label_index).view(-1)
    loss = criterion(out, edge_label)
    loss.backward()
    optimizer.step()
    return loss

@torch.no_grad()
def test(data):
    model.eval()
    z = model.encode(data.x, data.edge_index)
    out = model.decode(z, data.edge_label_index).view(-1).sigmoid()
    out_binary = np.array([1 if ele >=0.95 else 0 for ele in out.cpu().numpy()])
    label = data.edge_label.cpu().numpy()
    return roc_auc_score(label, out.cpu().numpy()), f1_score(label, out_binary), precision_score(label, out_binary), recall_score(label, out_binary)

In [ ]:
def training_visualization(auc, f1, precision, recall) :
    x_range = np.array(range(1, epochs+1))
    
    fig = make_subplots(
        rows=1, cols=2,
        subplot_titles=('validation', 'test')
    )

    fig.add_trace(
        go.Scatter(x=x_range, y=auc[0], mode='lines', name='val_auc'),
        row=1, col=1
    ) 
    
    fig.add_trace(
        go.Scatter(x=x_range, y=f1[0], mode='lines', name='val_f1'),
        row=1, col=1
    )

    fig.add_trace(
        go.Scatter(x=x_range, y=precision[0], mode='lines', name='val_precision'),
        row=1, col=1
    )

    fig.add_trace(
        go.Scatter(x=x_range, y=recall[0], mode='lines', name='val_recall'),
        row=1, col=1
    )

    fig.add_trace(
        go.Scatter(x=x_range, y=auc[1], mode='lines', name='test_auc'),
        row=1, col=2
    ) 
    
    fig.add_trace(
        go.Scatter(x=x_range, y=f1[1], mode='lines', name='test_f1'),
        row=1, col=2
    )

    fig.add_trace(
        go.Scatter(x=x_range, y=precision[1], mode='lines', name='test_precision'),
        row=1, col=2
    )

    fig.add_trace(
        go.Scatter(x=x_range, y=recall[1], mode='lines', name='test_recall'),
        row=1, col=2
    )

    fig.update_layout(height=500, width=900,
                  title_text="link prediction loss function graphs")
    fig.show()
    plot(fig, filename = 'training_graph', auto_open=True)


In [ ]:
best_val_auc = final_test_auc = 0
final_test_f1 = final_test_precision = final_test_recall = 0

# visualize training process 
# add all validation and test metrics into list to draw graph 
val_auc_list, val_f1_list, val_precision_list, val_recall_list = [],[],[],[]
test_auc_list, test_f1_list, test_precision_list, test_recall_list = [],[],[],[]

# training parts
for epoch in range(1, epochs+1):
    loss = train()
    val_auc, val_f1, val_precision, val_recall = test(val_data)
    test_auc, test_f1, test_precision, test_recall = test(test_data)

    # for visualization 
    val_auc_list.append(val_auc); val_f1_list.append(val_f1); val_precision_list.append(val_precision); val_recall_list.append(val_recall)
    test_auc_list.append(test_auc); test_f1_list.append(test_f1); test_precision_list.append(test_precision); test_recall_list.append(test_recall)
    
    # find best metrics when training 
    if val_auc > best_val_auc:
        best_val = val_auc
        final_test_auc = test_auc
        final_test_f1 = test_f1
        final_test_precision = test_precision
        final_test_recall = test_recall 

    if epoch % 10 == 0 :
        print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Val: {val_auc:.4f}, '
            f'Test: {test_auc:.4f}')

# for visualization 
auc_list = [val_auc_list, test_auc_list]
f1_list = [val_f1_list, test_f1_list]
precision_list = [val_precision_list, test_precision_list]
recall_list = [val_recall_list, test_recall_list]

print('\n')
print('Final Result')
print(f'Final Test AUC: {final_test_auc:.4f}')
print(f'Final Test F1 score: {final_test_f1:.4f}')
print(f'Final Test Precision: {final_test_precision:.4f}')
print(f'Final Test Recall: {final_test_recall:.4f}')


# conduct link prediction in currnet network 
# output form is adjacency matrix with logits 
z = model.encode(data.x, data.edge_index)
pred_logit = model.decode_all(z)

In [ ]:
# visualization traing process 
training_visualization(auc_list, f1_list, precision_list, recall_list)

In [ ]:
# Extract repo-topic heterogeneous network
# Conduct dichotomize
pred_network = pred_logit.sigmoid()

#pred_network = torch.where(pred_network>0.92, 1, 0)

original network has 78,683 edges 

- When the edge threshold is 0.8, the number of created edges is 302,876, which is very large compared to the original network edges.
- When the edge threshold is 0.9, the number of created edges is 98,041, which is little large compared to the original network edges.
- When the edge threshold is 0.95, the number of created edges is 44,948, which is quite small compared to the original network edges.
- When the edge threshold is 0.92, the number of created edges is 75,201, which is similar compared to the original network edges.

In [ ]:
# change network from tensor to pandas dataframe
pred_network = pred_network.detach().numpy()
pred_network = pd.DataFrame(pred_network, columns=node_data, index=node_data)

# remove diagonal term
for i in range(len(pred_network)) :
    pred_network.iloc[i, i] = 0

In [ ]:
pred_network.to_csv('data/network/predicted_network.test.csv')

# 4. VGAE based link prediction

The community detection method detects too many technologies, making it difficult to accurately detect technologies.

Therefore, we intend to derive about 10-15 technology clusters through clustering.

We use a graph autoencoder to learn the characteristics of the network.

Based on this, future link prediction is also performed.

In [6]:
class VariationalGCNEncoder(torch.nn.Module) : 
    def __init__(self, in_channels, out_channels) :
        super().__init__() 
        self.conv1 = GCNConv(in_channels, 2 * out_channels)
        self.conv_mu = GCNConv(2 * out_channels, out_channels)
        self.conv_logstd = GCNConv(2 * out_channels, out_channels)

    def forward(self, x, edge_index, edge_weight) : 
        x = self.conv1(x, edge_index, edge_weight).relu()
        return self.conv_mu(x, edge_index, edge_weight), self.conv_logstd(x, edge_index, edge_weight)

In [7]:
epochs = 200
in_channels, out_channels = data.x.size(1), 16

In [8]:
def train() :
    model.train()
    optimizer.zero_grad()
    z = model.encode(train_data.x, train_data.edge_index, train_data.edge_attr)
    loss = model.recon_loss(z, train_data.pos_edge_label_index)
    loss = loss + (1 / train_data.num_nodes) * model.kl_loss()
    loss.backward()
    optimizer.step()

    return float(loss)


@torch.no_grad()
def test(data) :
    model.eval()
    z = model.encode(data.x, data.edge_index, data.edge_attr)
    return model.test(z, data.pos_edge_label_index, data.neg_edge_label_index)

In [9]:
model_container = []
auc_container = []

for tries in range(1, 11) : 
    model = VGAE(VariationalGCNEncoder(in_channels, out_channels)).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

    print('=============================={}=============================='.format(tries))
    for epoch in range(1, epochs + 1):
        loss = train()
        auc, ap = test(test_data)
        if tries == 10 :
            auc_container.append(auc)
        if epoch % 50 == 0  :
            print(f'Epoch: {epoch:03d}, AUC: {auc:.4f}, AP: {ap:.4f}')

    model_container.append(model)

==============================1==============================
Epoch: 050, AUC: 0.8829, AP: 0.8989
Epoch: 100, AUC: 0.9128, AP: 0.9186
Epoch: 150, AUC: 0.9387, AP: 0.9406
Epoch: 200, AUC: 0.9532, AP: 0.9538
==============================2==============================
Epoch: 050, AUC: 0.8889, AP: 0.9032
Epoch: 100, AUC: 0.9303, AP: 0.9346
Epoch: 150, AUC: 0.9563, AP: 0.9575
Epoch: 200, AUC: 0.9628, AP: 0.9632
==============================3==============================
Epoch: 050, AUC: 0.8847, AP: 0.9005
Epoch: 100, AUC: 0.8944, AP: 0.9055
Epoch: 150, AUC: 0.9163, AP: 0.9232
Epoch: 200, AUC: 0.9345, AP: 0.9372
==============================4==============================
Epoch: 050, AUC: 0.9019, AP: 0.9133
Epoch: 100, AUC: 0.9325, AP: 0.9365
Epoch: 150, AUC: 0.9494, AP: 0.9509
Epoch: 200, AUC: 0.9586, AP: 0.9589
==============================5==============================
Epoch: 050, AUC: 0.8950, AP: 0.9081
Epoch: 100, AUC: 0.9239, AP: 0.9290
Epoch: 150, AUC: 0.9399, AP: 0.9423
Epoch:

In [10]:
x_range = np.array(range(1, epochs+1))

fig = make_subplots(
)

fig.add_trace(
    go.Scatter(x=x_range, y=auc_container, mode='lines'),
    row=1, col=1
)
    
fig.show()
plot(fig, filename = 'training_graph', auto_open=True)

'https://plotly.com/~injokim/19/'

In [11]:
def ensemble_network(voting_method) : 
    # voting method has 2 conditions 'soft' or 'hard'
    
    assert voting_method == 'soft' or voting_method == 'hard'
    edge_threshold = 0.98

    # soft voting 
    if voting_method == 'soft' :
        new_network = torch.zeros((n_topic, n_topic))

        for i in range(10) : 
            z = model_container[i].encode(data.x, data.edge_index, data.edge_attr)
            new_network += model_container[i].decoder.forward_all(z)

        new_network /= 10
        new_network = torch.where(new_network>= edge_threshold, 1, 0)
        mean_z = None


    # hard voting
    if voting_method == 'hard' :
        new_network = torch.zeros((n_topic, n_topic))
        
        for i in range(10) :
            z = model_container[i].encode(data.x, data.edge_index, data.edge_attr)
            new_network += torch.where(model_container[i].decoder.forward_all(z)>= edge_threshold, 1, 0)

            if i == 0 :
                mean_z = torch.zeros(z.size())
            mean_z += z

        new_network = torch.where(new_network >= 10, 1, 0)
        mean_z /= 10

    # remove diagonal term
    for i in range(new_network.size(1)) : 
        new_network[i,i] = 0

    return new_network, mean_z

In [12]:
soft_voting_network, _ = ensemble_network('soft')
hard_voting_network, z = ensemble_network('hard')

G_soft = nx.from_numpy_matrix(soft_voting_network.detach().numpy())
G_hard = nx.from_numpy_matrix(hard_voting_network.detach().numpy())

print(nx.info(G_soft))
print(nx.info(G_hard))

Graph with 6463 nodes and 35104 edges
Graph with 6463 nodes and 20722 edges


In [ ]:
pred_network = pd.DataFrame(hard_voting_network.detach().numpy(), index=node_data, columns=node_data)

In [ ]:
pred_network.to_csv('data/network/predicted_network/predicted_coword.csv')

# 5. Predicted network analysis 

## 5-1. Network clustering

Not clustering in all nodes.   


If clustering in specific nodes(in original network), maybe it will return different output

In [90]:
def node_selector(type) : 
    assert type in ['original', 'predicted']
    
    if type == 'original' :
        node = pd.read_csv('data/network/coword_node/coword_node.csv')
        node = list(node[node.Degree>=3].Id)

        
        
    elif type == 'predicted' : 
        node = pd.read_csv('add after')
        node = list(node.Id)


    node_index_dict = {}
    for i, node1 in enumerate(node_data) :
        for _, node2 in enumerate(node) : 
            if node1 == node2 : 
                node_index_dict[node1] = i 
                continue

    valualble_index = list(node_index_dict.values())
    new_z = z[valualble_index, : ]
    
    return list(node_index_dict.keys()), new_z

In [94]:
# Extract only nodes that exist in the current network from the entire vector
current_nodes, current_vectors = node_selector('original')

# clustering using kmeans
embedding_vector = current_vectors.detach().numpy()
kmeans = KMeans(n_clusters=15, random_state=1121).fit(embedding_vector)
clusters = kmeans.labels_


# assign cluster at nodes
repo_cluster = {row : clusters[row_idx] for row_idx, row in enumerate(current_nodes)}
repo_cluster = pd.DataFrame.from_dict(repo_cluster, orient='index')

# change data type to dataframe 
repo_cluster.reset_index(inplace=True)
repo_cluster.columns = ['Id', 'cluster']

In [95]:
repo_cluster.to_csv('data/network/cluster/coword_cluster.csv', index=False)

## 5-2. Visualize embeded nodes

In [96]:
# Extract only nodes that exist in the current network from the entire vector
current_nodes, current_vectors = node_selector('original')

# Reduce vectors to 2D for visualization
current_tsne = TSNE(n_components=2).fit_transform(current_vectors.detach().numpy())

# visualize 
fig = go.Figure()
fig.add_trace(go.Scatter(x=current_tsne[:,0], y=current_tsne[:,1], text=current_nodes, textposition="bottom center",
                    marker=dict(color=clusters, colorscale='curl'), textfont=dict(size=7), mode='markers+text'))

fig.update_layout(width=1600, height=1000)

fig.show()

## 5-3. Extract sub-network

In [ ]:
# extract in original network
for i in np.unique(repo_cluster.cluster) :
    word_per_cluster = list(repo_cluster[repo_cluster.cluster==i]['Id'])
    small_network = original.loc[word_per_cluster, word_per_cluster]
    for j in range(len(small_network)) :
        small_network.iloc[j, j] = 0
    small_network.to_csv('data/network/normalized_coword_subnetwork/{}_subnetwork.csv'.format(i))